- __Contour approximation__ is predicated on the assumption that a curve can be approximated by a series of short line segments. This leads to a resulting approximated curve that consists of a subset of points that were defined by the original curve.
- Contour approximation is actually already implemented in OpenCV via the __cv2.approxPolyDP  method__.
- Common values for the second parameter to __cv2.approxPolyDP__  are normally in the range of __1-5%__ of the original contour perimeter.

In [1]:
import cv2

class ShapeDetector:
    
    def __init__(self):
        pass
    
    def detect(self, c):
        # initialize the shape name and approximate the contour
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        
        # if the shape is a triangle, it will have 3 vertices
        if len(approx) == 3:
            shape = "triangle"
        
        # if the shape has 4 vertices, it is either a square or
        # a rectangle
        elif len(approx) == 4:
            # compute the bounding box of the contour and use the
            # bounding box to compute the aspect ratio
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h) 
            
            # a square will have an aspect ratio that is approximately
            # equal to one, otherwise, the shape is a rectangle
            shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"
        
        # if the shape is a pentagon, it will have 5 vertices
        elif len(approx) == 5:
            return "pentagon"
        
        # otherwise, we assume the shape is a circle
        else:
            shape = "circle"
        
        return shape

In [3]:
import imutils

#load the image and resize it to a smaller factor so that
# the shapes can be approximated better
image = cv2.imread("shapes_and_colors.jpg")
resized = imutils.resize(image, width = 300)
ratio = image.shape[0] / float(resized.shape[0])

gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]

cnts, hr = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

sd = ShapeDetector()

for c in cnts:
    # detect the name of the shape using only the contour, then compute the center of the contour
    shape = sd.detect(c)
    M = cv2.moments(c)
    cX = int((M["m10"] / M["m00"]) * ratio )
    cY = int((M["m01"] / M["m00"]) * ratio )
    
    # multiply the contour (x, y)-coordinates by the resize ratio,
    # then draw the contours and the name of the shape on the image
    c = c.astype("float")
    c *= ratio
    c = c.astype("int")
    cv2.drawContours(image, c, -1, (0, 255, 0), 2)
    cv2.putText(image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    cv2.imshow("Image", image)
    cv2.waitKey(1000)

cv2.destroyWindow("Image")

- Since we are processing the contours extracted from the resized image (rather than the original image), we need to multiply the contours and center (x, y)-coordinates by our resize ratio  (Lines 43-45). This will give us the correct (x, y)-coordinates for both the contours and centroid of the original image.